# Kubeflow pipeline
This is a fairly simple pipeline, containing sequential steps:

1. Update data - implemented by lightbend/recommender-data-update-publisher:0.2 image
2. Run model training. Ideally we would run TFJob, but due to the current limitations for pipelines, we will directly use an image implementing training lightbend/ml-tf-recommender:0.2
3. Update serving model - implemented by lightbend/recommender-model-publisher:0.2

# Setup

In [1]:
!pip install kubernetes --upgrade --user
!pip install kfp --upgrade --user


import kfp  # the Pipelines SDK.  This library is included with the notebook image.
from kfp import compiler
import kfp.dsl as dsl
import kfp.notebook
from kubernetes import client as k8s_client

     |████████████████████████████████| 1.5MB 3.5MB/s eta 0:00:01
ERROR: kfp 0.1.39 has requirement kubernetes<=9.0.0,>=8.0.0, but you'll have kubernetes 10.0.1 which is incompatible.
  Found existing installation: kubernetes 9.0.0
    Uninstalling kubernetes-9.0.0:
      Successfully uninstalled kubernetes-9.0.0
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: kfp in ./.local/lib/python3.6/site-packages (0.1.39)
  Using cached https://files.pythonhosted.org/packages/00/f7/4f196c55f1c2713d3edc8252c4b45326306eef4dc10048f13916fe446e2b/kubernetes-9.0.0-py2.py3-none-any.whl
  Found existing installation: kubernetes 10.0.1
    Uninstalling kubernetes-10.0.1:
      Successfully uninstalled kubernetes-10.0.1
You should consider upgrading via the 'pip install --upgrade pip' command.


# Create/Get an Experiment in the Kubeflow Pipeline System
The Kubeflow Pipeline system requires an "Experiment" to group pipeline runs. You can create a new experiment, or call client.list_experiments() to get existing ones.

In [2]:
client = kfp.Client()
client.list_experiments()
#exp = client.create_experiment(name='mdupdate')
exp = client.get_experiment(experiment_name ='mdupdate')

# Define a Pipeline
Authoring a pipeline is like authoring a normal Python function. The pipeline function describes the topology of the pipeline.

Each step in the pipeline is typically a ContainerOp --- a simple class or function describing how to interact with a docker container image. In the pipeline, all the container images referenced in the pipeline are already built.

In [3]:
@dsl.pipeline(
  name='Recommender model update',
  description='Demonstrate usage of pipelines for multi-step model update'
)
def recommender_pipeline():
    # Load new data
  data = dsl.ContainerOp(
      name='updatedata',
      image='lightbend/recommender-data-update-publisher:0.2') \
    .add_env_variable(k8s_client.V1EnvVar(name='MINIO_URL',value='http://minio-service.kubeflow.svc.cluster.local:9000')) \
    .add_env_variable(k8s_client.V1EnvVar(name='MINIO_KEY', value='minio')) \
    .add_env_variable(k8s_client.V1EnvVar(name='MINIO_SECRET', value='minio123'))
    # Train the model
  train = dsl.ContainerOp(
      name='trainmodel',
      image='lightbend/ml-tf-recommender:0.2') \
    .add_env_variable(k8s_client.V1EnvVar(name='MINIO_URL',value='minio-service.kubeflow.svc.cluster.local:9000')) \
    .add_env_variable(k8s_client.V1EnvVar(name='MINIO_KEY', value='minio')) \
    .add_env_variable(k8s_client.V1EnvVar(name='MINIO_SECRET', value='minio123'))
  train.after(data)
    # Publish new model model
  publish = dsl.ContainerOp(
      name='publishmodel',
      image='lightbend/recommender-model-publisher:0.2') \
    .add_env_variable(k8s_client.V1EnvVar(name='MINIO_URL',value='http://minio-service.kubeflow.svc.cluster.local:9000')) \
    .add_env_variable(k8s_client.V1EnvVar(name='MINIO_KEY', value='minio')) \
    .add_env_variable(k8s_client.V1EnvVar(name='MINIO_SECRET', value='minio123')) \
    .add_env_variable(k8s_client.V1EnvVar(name='KAFKA_BROKERS', value='cloudflow-kafka-brokers.cloudflow.svc.cluster.local:9092')) \
    .add_env_variable(k8s_client.V1EnvVar(name='DEFAULT_RECOMMENDER_URL', value='http://recommendermodelserver.kubeflow.svc.cluster.local:8501')) \
    .add_env_variable(k8s_client.V1EnvVar(name='ALTERNATIVE_RECOMMENDER_URL', value='http://recommendermodelserver1.kubeflow.svc.cluster.local:8501'))
  publish.after(train)

# Compile pipeline

In [4]:
compiler.Compiler().compile(recommender_pipeline, 'pipeline.tar.gz')

# Submit an experiment run

In [5]:
run = client.run_pipeline(exp.id, 'pipeline1', 'pipeline.tar.gz')